# Data Dictionary:
## **1. Download Data**
> train_dataset    |   torchvision.datasets.mnist.FashionMNIST<br>
eval_dataset       |   torchvision.datasets.mnist.FashionMNIST

## **2. Prepare Data**


> train_images     |   numpy array (60000, 784)
<br>train_labels   |   numpy array (60000, )
<br>eval_images    |   numpy array (10000, 784)
<br>eval_labels    |   numpy array (10000, )
<br> standardized_train_images | numpy array (60000, 784)
<br> standardized_eval_images |   numpy array (10000, 784)




# **To-Do**
1. Test PCA with sharpened data
2. Test Feature selection on PCs

Add in F1 Score & Confusion matrix

In [0]:
# a = []
# while(1):
#     a.append('1')

# Importing of Libraries

In [1]:
import torchvision
from PIL import Image

import pandas as pd
import numpy as np
import scipy as sp
from scipy import ndimage
from scipy.stats import norm
import scipy.signal as sig

from sklearn.preprocessing import MinMaxScaler
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from skimage import io
from skimage.feature import hog
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import learning_curve
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import StratifiedKFold
from sklearn.feature_selection import RFECV

from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

import time, random, math


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
%matplotlib inline

# Functions

In [0]:
def print_time_taken(start_time):
    end_time = time.time()
    secs = end_time-start_time
    mins = math.floor(secs / 60)
    secs = secs % 60
    print('time taken:',str(mins), 'min ',str(secs),'s')

# Data Extraction

In [3]:
# download the Fashion MNIST training data
train_dataset = torchvision.datasets.FashionMNIST(root='./', train=True, download=True)

# download the Fashion MNIST evaluation data
eval_dataset = torchvision.datasets.FashionMNIST(root='./', train=False, download=True)

Extracting ./FashionMNIST/raw/train-images-idx3-ubyte.gz to ./FashionMNIST/raw


Extracting ./FashionMNIST/raw/train-labels-idx1-ubyte.gz to ./FashionMNIST/raw


Extracting ./FashionMNIST/raw/t10k-images-idx3-ubyte.gz to ./FashionMNIST/raw


Extracting ./FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to ./FashionMNIST/raw
Processing...
Done!


In [0]:
labels_dict = {0: "T-shirt/top",
               1: "Trouser",
               2: "Pullover",
               3: "Dress",
               4: "Coat",
               5: "Sandal",
               6: "Shirt",
               7: "Sneaker",
               8: "Bag",
               9: "Ankle boot"}

# Data Preparation

**Reshape data from 28x28 into 784**

In [0]:
train_images = train_dataset.data.numpy().reshape(-1,28*28)
train_labels = train_dataset.targets.data.numpy()

In [6]:
train_images.shape

(60000, 784)

In [7]:
train_labels.shape

(60000,)

In [0]:
eval_images = eval_dataset.data.numpy().reshape(-1,28*28)
eval_labels = eval_dataset.targets.data.numpy()

In [9]:
eval_images.shape

(10000, 784)

In [10]:
eval_labels.shape

(10000,)

**Standardize data**

In [0]:
standardized_train_images = StandardScaler().fit_transform(train_images)
standardized_eval_images = StandardScaler().fit_transform(eval_images)

# Support Vector Machine

In [0]:
# optimal number for random_state
random_seed = 42

**Run SVM on Untreated Data**

In [0]:
# svm = SVC(kernel='rbf', random_state=random_seed)

In [0]:
# cvs = cross_val_score(svm, train_images, train_labels,scoring='precision_macro')
# print np.mean(cvs)

In [0]:
# svm.fit(train_images,train_labels)

In [0]:
# y_pred_1 = svm.predict(eval_images)

# print('Model classification accuracy: {}%'.format(str(metrics.accuracy_score(eval_labels, y_pred_1) * 100)))
# f1 = f1_score(eval_labels, y_pred_1,average='macro')
# print('F1 score: %f' % f1)

**Run SVM on Standardized Data**

In [0]:
svm = SVC(kernel='rbf', random_state=random_seed)

In [27]:
cvs = cross_val_score(svm, train_images, train_labels,scoring='precision_macro')
print(np.mean(cvs))

0.8890649801213758


In [28]:
svm.fit(standardized_train_images,train_labels)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=42, shrinking=True, tol=0.001,
    verbose=False)

In [29]:
y_pred_2 = svm.predict(standardized_eval_images)

print('Model classification accuracy: {}%'.format(str(metrics.accuracy_score(eval_labels, y_pred_2) * 100)))
f1 = f1_score(eval_labels, y_pred_2,average='macro')
print('F1 score: %f' % f1)

Model classification accuracy: 88.36%
F1 score: 0.882864


**Run SVM with PCA**

In [0]:
pca = PCA()

pca.n_components = 11

pca_train_images = pca.fit_transform(standardized_train_images)
pca_eval_images = pca.fit_transform(standardized_eval_images)

In [0]:
svm = SVC(kernel='rbf', C=10, random_state=random_seed)

In [23]:
%%time

svm.fit(pca_train_images,train_labels)

CPU times: user 39.5 s, sys: 114 ms, total: 39.6 s
Wall time: 39.6 s


SVC(C=10, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=42, shrinking=True, tol=0.001,
    verbose=False)

In [24]:
y_pred_3 = svm.predict(pca_eval_images)

print('Model classification accuracy: {}%'.format(str(metrics.accuracy_score(eval_labels, y_pred_3) * 100)))
f1 = f1_score(eval_labels, y_pred_3,average='macro')
print('F1 score: %f' % f1)

Model classification accuracy: 81.54%
F1 score: 0.813687


**SVM GridSearchCV**

Testing Linear C Values

In [0]:
c_values = [0.0001, 0.01, 0.1, 1, 10, 100]
gamma_values = ['auto','scale']
degree_values = np.arange(1, 3, 1)
# kernel_types = ['linear','rbf','poly','sigmoid']
# param_grid = dict(kernel=kernel_types, gamma=gamma_range, C=c_range,degree=degree_values)
# tuning_param_grid = [{'kernel':['linear'], 'C':c_values}]
tuning_param_grid = [{'kernel':['rbf'], 'C':c_values, 'gamma':gamma_values}]
                    #  {'kernel':['poly'],'degree':degree_values, 'C':c_values, 'gamma':gamma_values}]
                    #  {'kernel':['sigmoid'], 'C':c_values, 'gamma':gamma_values}]
svm = SVC()
cv = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=random_seed)

In [0]:
start = time.time()

In [33]:
%%time
grid = GridSearchCV(svm, param_grid=tuning_param_grid, cv=cv, verbose=10, n_jobs=-1)
grid.fit(pca_train_images,train_labels)

Fitting 5 folds for each of 12 candidates, totalling 60 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done   5 tasks      | elapsed: 15.0min
[Parallel(n_jobs=-1)]: Done  10 tasks      | elapsed: 22.3min
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed: 29.4min
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed: 36.4min
/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed: 44.2min
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed: 52.7min
[Parallel(n_jobs=-1)]: Done  53 tasks      | elapsed: 63.0min
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 67.5min remaining:    0.0s
[Parallel(n_jobs=-1)]: Done  60 out of  60 | elapsed: 67.5min finished


CPU times: user 1min 3s, sys: 404 ms, total: 1min 3s
Wall time: 1h 8min 34s


In [37]:
print ("The best classifier is: " , grid.best_estimator_)
# print ('Score of best classifier :', grid.score())

The best classifier is:  SVC(C=100, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


In [0]:
# grid.best_param.get("kernel")
# grid.best_param.get("gamma")
# grid.best_param.get("C")
# grid.best_param.get("degree")
# grid.best_param

In [0]:
# for param, score in zip(grid.cv_results_['params'], grid.cv_results_['mean_test_score']):
#     print(param, score)

**Recursive feature elimination**


m = RFECV(RandomForestClassifier(), scoring='accuracy')
m.fit(X, y)